In [2]:
import pandas as pd
import importlib

#### 1. Import the raw text data and generate the corpus

- The raw text data is eaxtracted from the insurance documents using te pdfminer
- The corpus is stored as a list where each element is a string containing the document content

In [ ]:
import src.raw_data as raw_data
importlib.reload(raw_data)

<module 'src.raw_data' from 'c:\\Users\\felix\\Documents\\RAG\\notebooks\\..\\src\\raw_data.py'>

In [ ]:
raw_data = raw_data.DataLoader(directory="C:/Users/felix/Documents/RAG/data/pdf")

In [ ]:
corpus = raw_data.gen_corpus()

#### 2. Preprocess the raw corpus

- The corpus is preprocessed (data cleaning etc.)
- Further, the each document is divided into chunks
- The chunks are stored in a dataframe, df_chunks
- The preprocessed corpus is stored as a list, preprocessed_corpus, for the later topic modelling

In [ ]:
import src.preprocessing as preprocessing
importlib.reload(preprocessing)

preprocessor = preprocessing.Preprocessor(corpus=corpus)

In [ ]:
preprocessor.clean_data()

preprocessed_corpus = preprocessor.corpus

In [ ]:
df_chunks = preprocessor.trigger_preprocessing()

#### 3. Create embeddings

- For each chunk a embedding is generated using OpenAIs latest embedding model

In [ ]:
import src.embedding as embedding
importlib.reload(embedding)

embedder = embedding.Embedder(df_chunks=df_chunks)

In [ ]:
embedder.trigger_embedder()

In [63]:
df_chunks = embedder.df_chunks

#### 4. Topic modelling

- We use tfidf vectorization to retrieve the top topics to each document and add this information is a column to each chunk
- The topics shall capture more broad context information of the chunk

In [11]:
import src.topicmodelling as topicmodelling
importlib.reload(topicmodelling)

topicmodelling = topicmodelling.TopicModeller(corpus=preprocessed_corpus)

In [12]:
top_words_corpus = topicmodelling.trigger_tfidf()

Document 1:
privathaftpflichtversicherung
euro
angebot
million
telefon
ändern
angabe

Document 2:
glasversicherung
wohnung
angebot
telefon
mehrfamilienhaus
ändern
angabe

Document 3:
euro
hausratversicherung
weils
fortsetzung
selbstbeteiligung
angebot
wohnung

Document 4:
uhr
vermittlerin
hukcoburg
rheinbach
telefon
petra
tillmanns

Document 5:
uhr
vermittlerin
hukcoburg
telefon
persönlich
petra
tillmanns

Document 6:
unterlage
kopie
ablauf
zurückverlang
dahin
ginal
gang

Document 7:
unterlage
kopie
ablauf
zurückverlang
dahin
ginal
gang

Document 8:
ständig
bewohnt
sache
abweichend
wohnung
versichern
sonstig

Document 9:
euro
uhr
privathaftpflichtversicherung
vertrag
antrag
telefon
anzeigepflicht

Document 10:
uhr
antrag
glasversicherung
telefon
vertrag
anzeigepflicht
antrags

Document 11:
euro
beispiel
schutz
antrag
inklusive
naturgefahr
uhr

Document 12:
unternehmensgruppe
hukcoburg
gmbh
einwilligung
gesellschaft
sowie
daten

Document 13:
sache
versichert
gelten
wohnung
müssen
versic

In [15]:
doc_id_list = [id for id in range(28)]



df_topic = pd.DataFrame(doc_id_list, columns=['doc_id'])

df_topic['topics'] = top_words_corpus

In [16]:
df_chunks = df_chunks.merge(df_topic, how='left', on='doc_id')

In [27]:
df_chunks.to_csv('../data/embeddings/embeddings_090323.csv', index=False)

#### 5. Generate Answers to Various Questions

The following steps lead to an answer generating

In [ ]:
import src.answering as answering
importlib.reload(answering)

answering = answering.AnswerGenerator(database=df_chunks)

In [ ]:
answer = answering.trigger_answer(question="Ist ein Einbruchsdiebstahl versichert im Rahmen der Hausratversicherung?")
answer

[('Das gilt auch für den Diebstahl der Schlüssel.Besonderheiten in der Außenversicherung: b. Für Schäden durch Einbruchdiebstahl haben Sie nur dann Außenversi-cherungsschutz, wenn die Voraussetzungen einer der Varianten nach a. entsprechend erfüllt sind.2.3.2 Vandalismus nach einem Einbruch Vandalismus liegt vor, wenn der Täter in die Wohnung eindringt und versicherte Sachen vorsätzlich zerstört oder beschädigt.', ['sache', 'versichert', 'gelten', 'wohnung', 'müssen', 'versichern', 'entschädigung']), ('Deswegen ist der einfache Diebstahl auch nur in bestimmten Fällen versichert.Der Ein-bruchdiebstahl dagegen gehört zu den üblicherweise versicherten Gefahren. 8. TextformFür manche rechtserhebliche Erklärungen müssen Sie die Textform einhalten.', ['sache', 'versichert', 'gelten', 'wohnung', 'müssen', 'versichern', 'entschädigung']), ('Voraussetzung ist, dass sich der Einbruchdiebstahl auf einer Reise oder Fährüberfahrt ereignet hat.Diesen Versicherungsschutz haben Sie auch, wenn versiche

'Ja, Einbruchdiebstahl ist grundsätzlich im Rahmen der Hausratversicherung versichert, es können jedoch bestimmte Voraussetzungen oder Bedingungen gelten.'

In [ ]:
questions = ['Sind Gartenmöbel versichert?',
             'Inwiefern sind Gartenmöbel versichert?',
             'Sind Gartenmöbel im Rahmen der Hausratversicherung mit versichert?',
             'Welcher Versicherungsschutz besteht für Gartenmöbel?',
             'Ist ein Einbruchsdiebstahl versichert im Rahmen der Hausratversicherung?',
             'Wie hoch ist der Jahresbeitrag einer Haftpflichtversicherung?',
             'Wie viel kostet eine Haftpflichtversicherung im Jahr?',
             'Welche Sachen sind in der Glasversicherung versichert?',
             'Wie kann ich die Haftpflichtversicherung kündigen?',
             'Wie bin ich gegen Vandlismus geschützt?',
             'Wie schützt eine Hausratversicherung gegen Vandalismus?']

In [ ]:
for question in questions:
    answer = answering.trigger_answer(question=question)
    print(question, "\n", answer, "\n\n")

Inwiefern sind Gartenmöbel versichert? 
 Die Verischerung für Gartenmöbel ist nicht explizit in den vorgelegten Textabschnitten erwähnt. Jedoch könnten sie als bewegliche Sachen, die zu privaten Zwecken gemietet, geleast, geliehen oder verwahrt wurden, gelten, wie im ersten Textabschnitt beschrieben. Schäden an Möbeln und sonstigem Inventar von gemieteten oder geliehenen Räumen sind laut Textabschnitt vier allerdings ausgeschlossen. Es kann also sein, dass Gartenmöbel nur versichert sind, wenn spezieller Schutz, wie der im fünften Textabschnitt erwähnte Mobiliarschutz, vereinbart wurde. Die genaue Deckung hängt vom spezifischen Versicherungsvertrag ab. 


Sind Gartenmöbel im Rahmen der Hausratversicherung mit versichert?Wie hoch ist der Jahresbeitrag einer Haftpflichtversicherung? 
 Auf Basis der gegebenen Textabschnitte lässt sich die Frage nicht eindeutig beantworten, da keine spezifischen Informationen zu Gartenmöbeln in der Hausratversicherung oder zum Jahresbeitrag einer Haftpflic